In [17]:
# Here are the libraries we will need ...
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [18]:
data_column_name = pd.read_csv("C:\\Users\\Meeta\\Desktop\\Spring 2018\\Machine Learning\\Datasets\\Assignment_2_Data\\adult.names.txt", sep='\t', encoding = 'latin 1', header = None)

In [19]:
print (data_column_name)

                                                    0
0                                    age: continuous.
1   workclass: Private, Self-emp-not-inc, Self-emp...
2                                 fnlwgt: continuous.
3   education: Bachelors, Some-college, 11th, HS-g...
4                          education-num: continuous.
5   marital-status: Married-civ-spouse, Divorced, ...
6   occupation: Tech-support, Craft-repair, Other-...
7   relationship: Wife, Own-child, Husband, Not-in...
8   race: White, Asian-Pac-Islander, Amer-Indian-E...
9                                  sex: Female, Male.
10                          capital-gain: continuous.
11                          capital-loss: continuous.
12                        hours-per-week: continuous.
13  native-country: United-States, Cambodia, Engla...


In [54]:
data_column_name.iloc[0:, 0].value_counts()

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.                                                                                                                                                                                                                                                                                  1
capital-loss: continuous.                                                                                                                                                                                                                                                                                                                                                                                                                          1
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-i

In [20]:
train_data = pd.read_csv("C:\\Users\\Meeta\\Desktop\\Spring 2018\\Machine Learning\\Datasets\\Assignment_2_Data\\adultTrain.data", header = None)

In [ ]:
train_data.headers

In [21]:
test_data = pd.read_csv("C:\\Users\\Meeta\\Desktop\\Spring 2018\\Machine Learning\\Datasets\\Assignment_2_Data\\adultTest.data", header = None)

In [22]:

train_data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [23]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [55]:
train_data.iloc[:,1].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: 1, dtype: int64

In [25]:
train_data.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [26]:
print (train_data.shape)

(32561, 15)


In [30]:
train_data = train_data.dropna()

In [31]:
train_data.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [32]:
train_data.drop(train_data.columns[[2, 10, 11]], axis=1, inplace=True) 
train_data.columns

Int64Index([0, 1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14], dtype='int64')

In [ ]:
#train_data2 = pd.get_dummies(train_data, columns =['age','workclass','education','education-num','marital-status','occupation','relationship','race','sex','hours-per-week','native-country'])

In [ ]:
#New trial from beginning - for NN

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [3]:
INPUT_PATH = "C:\\Users\\Meeta\\Desktop\\Spring 2018\\Machine Learning\\Datasets\\Assignment_2_Data\\adultTrain.data"
headers = ["age", "workclass", "fnlwgt", "education", "education-num",
               "marital-status", "occupation", "relationship", "race", "sec", "capital-gain",
               "capital-loss", "hours-per-week", "native-country", "y"]
dataset = pd.read_csv(INPUT_PATH)
dataset.columns = headers

In [4]:
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sec,capital-gain,capital-loss,hours-per-week,native-country,y
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [5]:
dataset.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sec',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'y'],
      dtype='object')

In [ ]:
dataset['relationship'].value_counts()

In [7]:
dataset = dataset.dropna()

dataset.drop(dataset.columns[[2, 10, 11]], axis=1, inplace=True)

In [8]:
dataset2 = pd.get_dummies(dataset, columns =['age', 'workclass', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sec',
      'hours-per-week', 'native-country'])

In [9]:
dataset2.columns

Index(['y', 'age_17', 'age_18', 'age_19', 'age_20', 'age_21', 'age_22',
       'age_23', 'age_24', 'age_25',
       ...
       'native-country_ Portugal', 'native-country_ Puerto-Rico',
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia'],
      dtype='object', length=286)

In [10]:
X = dataset2.iloc[:,1:]
y = dataset2.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=0)

In [11]:
# Again we scale the data ... although this is probably not needed (data is binary)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
X_train.shape

(24420, 285)

In [13]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(23,23,23))

In [14]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(23, 23, 23), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [15]:
predictions = mlp.predict(X_test)

In [16]:
print(confusion_matrix(y_test,predictions))
mlp.score(X_test,y_test)

[[5455  738]
 [ 902 1045]]


0.79852579852579852

In [17]:
#Using KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=5)

In [19]:
KNN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [20]:
predictions = KNN.predict(X_test)

In [21]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
KNN.score(X_test,y_test)

[[5463  730]
 [ 889 1058]]


0.80110565110565113

In [22]:
# Using SVM

In [ ]:
from sklearn import svm

svc_linear = svm.SVC(kernel='linear', C=10)
svc_linear.fit(X_train, y_train)

In [ ]:
predicted= svc_linear.predict(X_test)
cnf_matrix = confusion_matrix(y_test, predicted)
print(cnf_matrix)
svc_linear.score(X_test,y_test)